<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dafaf2c3e3928d0a18817a5a762dce30a7dc299a94a89bb8d379be9e0de38d46
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 13:00:08
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 1.13 L (0.78%)
Current PnL: -18.23 L (-11.94%)
CY Booked + Current PnL: -6.10 L (-4.0%)
-------------------
Total profit:  2.83 L
Total loss:  -21.06 L
-------------------
Total Booked + Current PnL: 20.88 L (16.62%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 85.93 L (58.86%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.00,-6.16,76.0,X-LC,4.17,234593.0,24227.0,4551.0,1.14,11.52,1.94,13.68,37.0,5.32,1.62,29.94,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.00,112.17,67.0,M-SC,1.41,91445.0,-9332.0,9428.0,-2.53,-9.26,10.31,0.09,245.0,-0.99,0.63,20.14,OX40N,NTT,DURABLES
42,ITC,452.00,-36.66,64.0,X-LC,1.41,204842.0,4704.0,16183.0,0.24,2.35,7.90,10.44,4.0,0.29,1.42,8.04,X40,NTT,FMCG
2,ABBOTINDIA,35195.00,-9.84,39.0,X-MC,2.76,88980.0,-1698.0,16604.0,-0.37,-1.87,18.66,16.44,101.0,-0.10,0.62,17.93,X40,ATH,PHARMA
34,ICICIGI,2252.93,-12.91,65.0,X-MC,4.27,149894.0,13761.0,16818.0,0.33,10.11,11.22,22.47,91.0,0.82,1.04,24.40,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-12.81,67.0,X-LC,8.93,313896.0,-1946.0,114603.0,8.33,-0.62,36.51,35.67,5.0,-0.02,2.17,13.15,X40N,ATH,FMCG
62,SAIL,228.00,53.97,68.0,M-MC,10.15,242802.0,17840.0,149809.0,6.69,7.93,61.70,74.53,192.0,0.12,1.68,42.74,XY24,BTT,STEEL
82,VAIBHAVGBL,521.00,63.64,68.0,H-SC,3.88,141846.0,-40929.0,152002.0,4.83,-22.39,107.16,60.77,125.0,-0.27,0.98,27.05,XR,NTT,JEWELLERY
57,RECLTD,446.00,50.38,62.0,M-LC,5.46,210815.0,7645.0,34489.0,3.89,3.76,16.36,20.74,55.0,0.22,1.46,10.88,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,52.0,M-LC,11.38,210801.0,5725.0,106855.0,2.84,2.79,50.69,54.89,52.0,0.05,1.46,27.56,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,5211.76,-5.61,48.0,X-SC,0.41,107411.0,5407.0,38518.0,-3.74,5.30,35.86,43.06,122.0,0.14,0.74,23.32,X40N,ATH,MISC
70,STARHEALTH,761.00,17.34,53.0,H-SC,12.63,261365.0,-19758.0,152611.0,-2.87,-7.03,58.39,47.26,144.0,-0.13,1.81,40.03,XY24,NTT,INSURANCE
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88910.0,-39760.0,85060.0,-2.71,-30.90,95.67,35.20,219.0,-0.47,0.62,2.05,X40,NTT,FOOTWEAR
79,TTKPRESTIG,770.00,112.17,67.0,M-SC,1.41,91445.0,-9332.0,9428.0,-2.53,-9.26,10.31,0.09,245.0,-0.99,0.63,20.14,OX40N,NTT,DURABLES
11,BANDHANBNK,400.00,-7.85,58.0,H-SC,3.27,230802.0,-47757.0,305189.0,-1.92,-17.14,132.23,92.42,151.0,-0.16,1.60,33.38,XY24,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-29.46,65.0,H-SC,16.15,201637.0,1683.0,95556.0,0.73,0.84,47.39,48.63,115.0,0.02,1.40,12.49,XR,NTT,MISC
87,WIPRO,420.0,-14.25,47.0,M-LC,5.71,150648.0,-297.0,109747.0,0.25,-0.20,72.85,72.51,53.0,-0.00,1.04,5.81,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,112.17,67.0,M-SC,1.41,91445.0,-9332.0,9428.0,-2.53,-9.26,10.31,0.09,245.0,-0.99,0.63,20.14,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.06,55.0,H-SC,2.75,226827.0,-42840.0,79163.0,1.01,-15.89,34.90,13.47,138.0,-0.54,1.57,15.33,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,6.76,105194.0,-26641.0,68871.0,-0.15,-20.21,65.47,32.03,98.0,-0.39,0.73,17.83,OX40N,NTT,IT
18,CERA,9475.0,-24.34,42.0,H-SC,2.64,137977.0,-37926.0,79944.0,0.54,-21.56,57.94,23.89,149.0,-0.47,0.95,19.16,OX40N,NTT,CERAMICS
68,SIS,528.0,2038.86,46.0,H-SC,3.15,86318.0,-24714.0,48321.0,-1.01,-22.26,55.98,21.26,156.0,-0.51,0.60,16.62,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.0,-15.53,75.0,H-MC,16.23,216597.0,36789.0,75614.0,-0.18,20.46,34.91,62.52,88.0,0.49,1.5,61.52,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.97,68.0,M-MC,10.15,242802.0,17840.0,149809.0,6.69,7.93,61.70,74.53,192.0,0.12,1.68,42.74,XY24,BTT,STEEL
57,RECLTD,446.00,50.38,62.0,M-LC,5.46,210815.0,7645.0,34489.0,3.89,3.76,16.36,20.74,55.0,0.22,1.46,10.88,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,52.0,M-LC,11.38,210801.0,5725.0,106855.0,2.84,2.79,50.69,54.89,52.0,0.05,1.46,27.56,X5K,ATH,METALS
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132061.0,6764.0,52164.0,0.27,5.40,39.50,47.03,141.0,0.13,0.91,65.35,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,64.0,M-LC,9.41,166116.0,25276.0,19220.0,-0.66,17.95,11.57,31.59,66.0,1.32,1.15,49.19,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-15.53,75.0,H-MC,16.23,216597.0,36789.0,75614.0,-0.18,20.46,34.91,62.52,88.0,0.49,1.50,61.52,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.76,65.0,M-SC,3.38,129067.0,14269.0,79776.0,1.57,12.43,61.81,81.92,223.0,0.18,0.89,52.79,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132061.0,6764.0,52164.0,0.27,5.40,39.50,47.03,141.0,0.13,0.91,65.35,MH,ATH,POWER
37,INDIAMART,4810.62,-52.96,58.0,H-SC,6.80,131281.0,7945.0,123680.0,-0.14,6.44,94.21,106.72,119.0,0.06,0.91,31.33,AR,ATH,MISC
32,HINDZINC,730.22,29.16,52.0,M-LC,11.38,210801.0,5725.0,106855.0,2.84,2.79,50.69,54.89,52.0,0.05,1.46,27.56,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.03,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88910.0,-39760.0,85060.0,-2.71,-30.90,95.67,35.20,219.0,-0.47,0.62,2.05,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-20.48,37.0,X-MC,5.33,199395.0,-952.0,31465.0,0.15,-0.48,15.78,15.23,178.0,-0.03,1.38,31.62,X40N,NTT,REALTY
24,DMART,5391.45,-16.90,37.0,X-LC,9.88,101594.0,-992.0,27796.0,0.29,-0.97,27.36,26.13,38.0,-0.04,0.70,24.36,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-9.84,39.0,X-MC,2.76,88980.0,-1698.0,16604.0,-0.37,-1.87,18.66,16.44,101.0,-0.10,0.62,17.93,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-1.97,50.0,H-LC,1.06,157380.0,-28715.0,76188.0,1.22,-15.43,48.41,25.51,15.0,-0.38,1.09,15.98,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,64.0,X-LC,1.41,204842.0,4704.0,16183.0,0.24,2.35,7.90,10.44,4.0,0.29,1.42,8.04,X40,NTT,FMCG
53,PGHH,17907.65,-30.28,39.0,X-MC,2.58,201720.0,900.0,66890.0,0.37,0.45,33.16,33.76,80.0,0.01,1.40,5.67,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.33,51.0,X-MC,3.02,316029.0,-11962.0,118479.0,1.64,-3.65,37.49,32.48,92.0,-0.10,2.19,7.36,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,0.88,51.0,X-MC,3.88,211635.0,19893.0,17862.0,-1.23,10.37,8.44,19.69,99.0,1.11,1.46,17.88,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88910.0,-39760.0,85060.0,-2.71,-30.90,95.67,35.20,219.0,-0.47,0.62,2.05,X40,NTT,FOOTWEAR
10,BAJAJHFL,181.50,-14.27,48.0,X-MC,5.93,180859.0,-19647.0,116799.0,1.16,-9.80,64.58,48.45,90.0,-0.17,1.25,4.29,X40N,ATH,FINANCE
52,PAGEIND,51769.93,-27.05,42.0,X-MC,6.79,82370.0,-1140.0,21169.0,0.66,-1.37,25.70,23.98,82.0,-0.05,0.57,4.37,X40,ATH,APPARELS
21,COLPAL,3767.14,-3.45,49.0,X-MC,7.87,225200.0,-38165.0,151515.0,0.86,-14.49,67.28,43.04,84.0,-0.25,1.56,4.55,XY25,ATH,FMCG
3,ACC,3906.00,-36.77,57.0,X-MC,2.54,187440.0,-50511.0,203166.0,0.74,-21.23,108.39,64.15,174.0,-0.25,1.30,5.14,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.90,52.0,X-SC,38.19,54651.0,-10355.0,161281.0,1.34,-15.93,295.11,232.18,198.0,-0.06,0.38,6.60,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.05,42.0,X-MC,6.79,82370.0,-1140.0,21169.0,0.66,-1.37,25.70,23.98,82.0,-0.05,0.57,4.37,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.10,47.0,X-SC,4.80,83002.0,-62158.0,138082.0,1.62,-42.82,166.36,52.31,136.0,-0.45,0.57,11.10,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.84,39.0,X-MC,2.76,88980.0,-1698.0,16604.0,-0.37,-1.87,18.66,16.44,101.0,-0.10,0.62,17.93,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88910.0,-39760.0,85060.0,-2.71,-30.90,95.67,35.20,219.0,-0.47,0.62,2.05,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.40,57.0,X-LC,11.29,288749.0,-57207.0,123902.0,0.45,-16.54,42.91,19.28,1.0,-0.46,2.00,6.75,X40,ATH,IT
40,INFY,2275.00,-17.09,61.0,X-LC,6.14,322631.0,9743.0,161928.0,0.97,3.11,50.19,54.87,3.0,0.06,2.23,11.65,X40,BTT,IT
42,ITC,452.00,-36.66,64.0,X-LC,1.41,204842.0,4704.0,16183.0,0.24,2.35,7.90,10.44,4.0,0.29,1.42,8.04,X40,NTT,FMCG
84,VBL,671.64,-12.81,67.0,X-LC,8.93,313896.0,-1946.0,114603.0,8.33,-0.62,36.51,35.67,5.0,-0.02,2.17,13.15,X40N,ATH,FMCG
1,ABB,7934.00,-39.28,57.0,H-LC,5.79,253584.0,-8035.0,127248.0,2.08,-3.07,50.18,45.57,7.0,-0.06,1.75,7.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7161.11,65.0,L-SC,7.63,79641.0,-14169.0,90791.0,0.41,-15.10,114.00,81.67,269.0,-0.16,0.55,54.23,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,27.94,81600.0,-31949.0,72004.0,-0.21,-28.14,88.24,35.27,268.0,-0.44,0.56,99.72,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.14,90375.0,-7605.0,29209.0,-0.15,-7.76,32.32,22.05,152.0,-0.26,0.63,31.37,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,112.17,67.0,M-SC,1.41,91445.0,-9332.0,9428.0,-2.53,-9.26,10.31,0.09,245.0,-0.99,0.63,20.14,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-37.76,65.0,M-SC,3.38,129067.0,14269.0,79776.0,1.57,12.43,61.81,81.92,223.0,0.18,0.89,52.79,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-195.45,73.0,M-SC,29.62,165384.0,16164.0,128024.0,-1.87,10.83,77.41,96.62,208.0,0.13,1.14,78.98,XY25,NTT,FINANCE
69,SONACOMS,806.63,-32.20,70.0,M-SC,7.98,84324.0,-16835.0,56834.0,-0.52,-16.64,67.40,39.54,202.0,-0.30,0.58,18.90,AR,ATH,AUTO
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132061.0,6764.0,52164.0,0.27,5.40,39.50,47.03,141.0,0.13,0.91,65.35,MH,ATH,POWER
12,BANKINDIA,190.00,-15.53,75.0,H-MC,16.23,216597.0,36789.0,75614.0,-0.18,20.46,34.91,62.52,88.0,0.49,1.50,61.52,XR,NTT,BANKS
5,ANGELONE,3033.00,16.33,63.0,X-SC,5.43,205451.0,14445.0,43247.0,-1.28,7.56,21.05,30.21,157.0,0.33,1.42,30.02,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.04
2,50,74.95


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.90
LC    32.43
MC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.03
X40      16.26
XY25     12.06
XR       11.41
X40N     10.65
AR        7.82
OX40N     6.49
X200      1.77
X5K       1.46
SR        1.08
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-LC    21.69
X-MC    18.73
M-SC    12.61
M-LC     5.11
H-LC     4.61
X-SC     4.50
H-MC     3.88
M-MC     1.68
L-SC     1.44
L-LC     1.02
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.55,-2.55,36.01
IT,12.52,-16.23,78.57
FINANCE,12.05,-8.48,57.65
MISC,6.84,-16.25,80.66
ELECTRICAL,5.80,-7.84,46.07
BANKS,5.65,-4.90,66.88
PAINTS,5.49,-16.21,33.53
INSURANCE,3.79,-1.01,38.67
AUTO,2.75,-44.27,104.48


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3232368.0,22
XR,1298122.0,14
AR,1157445.0,9
X40,808937.0,12
X40N,618601.0,9
OX40N,544043.0,9
XY25,439399.0,8
SR,265777.0,2
X5K,106855.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3312791.0,24
M-SC,1216748.0,15
X-MC,1172464.0,14
X-LC,1001699.0,13
X-SC,530336.0,6
H-MC,320067.0,3
H-LC,272416.0,3
M-LC,270311.0,4
L-SC,251945.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1154699.0      6
           AR         830950.0      5
           XR         766201.0      7
M-SC       XY24       695664.0      6
X-MC       XY24       572583.0      4
X-LC       X40        441637.0      5
           XY24       303530.0      2
X-SC       X40N       283995.0      4
X-MC       X40        282240.0      6
H-SC       SR         265777.0      2
           OX40N      243000.0      3
H-LC       AR         203436.0      2
X-LC       X40N       186342.0      3
H-MC       XY24       175582.0      1
X-MC       XY25       169377.0      2
L-SC       XR         162795.0      2
X-SC       XY24       161281.0      1
M-MC       XY24       149809.0      1
X-MC       X40N       148264.0      2
M-SC       OX40N      143022.0      4
           XY25       128024.0      1
           XR         126979.0      2
           AR         123059.0      2
M-LC       XR         109747.0      1
           X5K        106855.0      1
L-SC       OX40N       89150.0      1
X-SC       X40         85060.0      1
H-MC       XR          75614.0      1
X-LC       XY25        70190.0      3
H-LC       X200        68980.0      1
H-MC       OX40N       68871.0      1
L-MC       XR          56786.0      1
H-SC       MH          52164.0      1
L-LC       XY25        37319.0      1
M-LC       XY25        34489.0      1
           XY24        19220.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
